#Import Tweets Data 


1.   Import Tweets from sntwitter API (https://github.com/JustAnotherArchivist/snscrape)

2.   Clean up data - Remove special characters, emojis, memes in each tweets

In [ ]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as mlpt
import csv
!pip install snscrape
import datetime as dt
import snscrape.modules.twitter as sntwitter

In [ ]:
tweets = []
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#Black Lives Matter  since:2022-01-01','corona lang:en').get_items()):
    tweets.append([tweet.date,  tweet.content])

In [ ]:
tweets = pd.DataFrame(tweets, columns=['Date', 'Tweets'])
tweets

In [ ]:
data=tweets
cdata=pd.DataFrame(columns=['Date','Tweets'])
index=0
for index,row in data.iterrows():
    stre=row["Tweets"]
    #the following cleaning steps are not the standard. You may generate your own ways of cleaning.
    my_new_string = re.sub('#\w+', '', stre) #remove the hashtags
    my_new_string = re.sub('http\S+', '', my_new_string) #remove the links
    my_new_string = re.sub('\n', '', my_new_string) #remove \n
    cdata.sort_index()
    cdata.at[index,'Date']=row["Date"]
    cdata.at[index,'Tweets']=my_new_string
    index=index+1
tweets=cdata
tweets['Date'] = tweets['Date'].astype('datetime64[ns]')
tweets["Date"]=tweets["Date"].dt.date

In [ ]:
tweets.head()

#Sentiment Analysis for Tweets

Sentiment Analysis is a process of ‘computationally’ evaluating whether a piece of text is positive, negative or neutral. In our case, conducting sentiment analysis on tweets help determine the public's moods towards BLM(Black Lives Matter) topic.
1.   **VADER (Valence Aware Dictionary and Entiment Reasoner)** https://github.com/cjhutto/vaderSentiment

```
"VADER is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed
in social media. A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled
according to their semantic orientation aseither positive or negative. VADER not only tells about the Positivity
and Negativity score but also tells us about how positive or negative a sentiment is." (GreeksforGreeks)
```
2.   **Reference**
 
  Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

In [ ]:
!pip install vaderSentiment
!pip install pyspark

In [ ]:
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [ ]:
FullDataTw=sql.createDataFrame(tweets)
FullDataTw = FullDataTw.dropna()
FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
 #setting column names of Twitter dataset
CleanDF = FullDataTw.withColumnRenamed('Tweets', 'Tweet')
CleanDF = FullDataTw.withColumnRenamed('Date', 'Date_Time')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def senti_score_udf(sentence):
    snt = analyser.polarity_scores(sentence)
    return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])
func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['Tweets'])[0])  #withColumn function would generate the result with a new column name
CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['Tweets'])[1])
CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['Tweets'])[2])
CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['Tweets'])[3])
CleanDF.show(120)
CleanDF.toPandas().to_csv('sen.csv',index=False)

 **Categorize Sentiments**
Once getting the compound sentiment score for each Tweets, we put them into categories. 


  $$Negative \ Sentiment : -0.9<P_{comp}\leq -0.05$$
   
  $$Positive\  Sentiment: 0.05\leq P_{comp}<0.9$$

  $$Neutral \ Sentiment:  -0.05<P_{comp}<0.05 $$

  $$Extremely\ Negative \ Sentiment:  P_{comp}\leq -0.9 $$

  $$Extremely\ Postive \ Sentiment:  P_{comp}\geq 0.9 $$

In [ ]:
tw=pd.read_csv("sen.csv")
tw["length"]=""
i=0
# iterate through the csv file 
for val in tw["Tweets"]: 
  val = str(val) 
  tokens = val.split() 
  tw["length"][i]=len(tokens)
  i=i+1
tw=tw.drop(columns=["p_neg","p_neu","p_pos"])

In [ ]:
tw.loc[(tw['p_comp']>-0.05)&(tw['p_comp']<0.05),'sentiment']="neutral"
tw.loc[(tw['p_comp']>=0.05)&(tw['p_comp']<0.9),'sentiment']='positive'
tw.loc[(tw['p_comp']<=-0.05)&(tw['p_comp']>-0.9),'sentiment']='negative'
tw.loc[tw['p_comp']<=-0.9,'sentiment']='extremely negative'
tw.loc[tw['p_comp']>=0.9,'sentiment']='extremely positive'
tw

# Exercise2

#### Crawl data from twitter with another topic and do sentiment analysis on the data, after you get the results, try to use graphs to visualize the sentiment analysis results (any type of graph that could satisfy your purpose)